In [1]:
%pwd

'c:\\Text-summarization-End-to-End-project\\notebooks'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Text-summarization-End-to-End-project'

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [47]:
from src.TextSummarization.constants import *
from src.TextSummarization.utils.common import read_yaml, create_directories

In [48]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        """
        Returns a DataTransformationConfig object.
        """
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_configuration_config =  DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_configuration_config  

In [49]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
from src.TextSummarization.logging.Logging import logger
import os

In [50]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        try:
            input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

            with self.tokenizer.as_target_tokenizer():
                target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }

        except Exception as e:
            logger.error(e)
            raise e
        
    def convert(self):
        logger.info("Converting examples to features")
        dataset = load_from_disk(self.config.data_path)
        dataset_conv= dataset.map(self.convert_examples_to_features, batched = True)
        dataset_conv.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        logger.info("Finished converting examples to features")

In [51]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e 

    

[2024-05-02 14:08:43,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-02 14:08:43,051: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-02 14:08:43,053: INFO: common: created directory at: artifacts]
[2024-05-02 14:08:43,054: INFO: common: created directory at: artifacts/data_transformation]
[2024-05-02 14:08:44,118: INFO: 3458851253: Converting examples to features]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 95489.58 examples/s] 

[2024-05-02 14:08:51,264: INFO: 3458851253: Finished converting examples to features]
